# Eval using VLLM

In [ ]:
import wandb
from datasets import load_from_disk

In [ ]:
def load_from_artifact(at_address, at_type="dataset"):
    "Load a HF dataset from a W&B artifact"
    artifact = wandb.use_artifact(at_address, type=at_type)
    artifact_dir = artifact.download()
    return load_from_disk(artifact_dir)

In [ ]:
model_folder = "64yqh828_codellama7"
EVAL_DATASET_AT = 'capecape/wandbot_llm/wandbot_eval_dataset:latest'

In [ ]:
WANDB_PROJECT = "wandbot_llm"

In [ ]:
run = wandb.init(project=WANDB_PROJECT, job_type="vllm")

wandb: Currently logged in as: capecape. Use `wandb login --relogin` to force relogin


In [ ]:
from vllm import LLM, SamplingParams

In [ ]:
llm = LLM(model=model_folder)

INFO 10-18 20:00:30 llm_engine.py:72] Initializing an LLM engine with config: model='64yqh828_codellama7', tokenizer='64yqh828_codellama7', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=16384, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, seed=0)
INFO 10-18 20:00:30 tokenizer.py:31] For some LLaMA V1 models, initializing the fast tokenizer may take a long time. To reduce the initialization time, consider using 'hf-internal-testing/llama-tokenizer' instead of the original tokenizer.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 10-18 20:00:37 llm_engine.py:207] # GPU blocks: 739, # CPU blocks: 512


In [ ]:
eval_ds = load_from_artifact(EVAL_DATASET_AT)
eval_ds = eval_ds["train"]

wandb:   4 of 4 files downloaded.  


In [ ]:
max_new_tokens = 512
sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens=max_new_tokens)

In [ ]:
prompts = [sample["text"] for sample in eval_ds]

In [ ]:
[len(p) for p in prompts]

[10579,
 10469,
 8882,
 10431,
 8749,
 10198,
 10174,
 10348,
 10234,
 9650,
 10379,
 10402,
 10029,
 10091,
 9442,
 9709,
 9195,
 10209,
 10277,
 9074,
 9651,
 9998,
 8772,
 10131,
 8975,
 9456,
 9897,
 9503,
 9656,
 11637,
 10527,
 8915,
 9451,
 8722,
 9895,
 8831,
 9308,
 9776,
 10325,
 10324,
 10026,
 10123,
 10197,
 9554,
 9762,
 10710,
 8324,
 9975,
 8661,
 10323,
 9445,
 10325,
 10054,
 10328,
 10091,
 9369,
 10255,
 9732,
 10186,
 9309,
 10267,
 9887,
 10687,
 9162,
 8552,
 9981,
 9384,
 9895,
 9735,
 9525,
 8621,
 9534,
 10187,
 10269,
 10301,
 10465,
 7216,
 9650,
 10248,
 10133,
 8186,
 9668,
 9495,
 8558,
 10287,
 10188,
 9297,
 10365,
 9998,
 10299,
 9746,
 8660,
 10283,
 10595,
 10113,
 9924,
 9914,
 10344,
 10162,
 9519,
 8588,
 9309,
 9968,
 9369,
 10204,
 9506,
 9912,
 10441,
 10033,
 9999,
 10077,
 7328,
 9814,
 10473,
 9852,
 9494,
 8090,
 11143,
 10234,
 10030,
 10083,
 10307,
 10131,
 9639,
 10160,
 9401,
 10235,
 9334,
 10294,
 8577,
 9837,
 10410]

In [ ]:
outputs = llm.generate(prompts, sampling_params)

Processed prompts: 100%|██████████| 132/132 [11:23<00:00,  5.18s/it]


In [ ]:
generations = [[sample, output.outputs[0].text] for sample, output in zip(eval_ds, outputs)]

In [ ]:
len(generations)

132

In [ ]:
originals_cols = list(generations[0][0].keys())
table = wandb.Table(columns=originals_cols+["codellama7", "max_tokens"])
for gen in generations:
    d, code_answer = gen
    table.add_data(*d.values(), code_answer, max_new_tokens)

In [ ]:
wandb.log({"generations": table})

wandb: WARNING Serializing object of type str that is 146228 bytes


In [ ]:
wandb.finish()